# Tapovan dam Coherence Change Detection This notebook serves to demonstrate the use case of the usage of the Coherence Change Detection for quick analysis.
It has been applied on the flash flood event occurred in Tapovan Dam, Uttarakhand (India) on 7th Feb 2021d

In [ ]:
#@title libraries for input data
try:
  import ipywidgets as widgets
except:
  import ipywidgets as widgets
import zipfile
import os
import glob
try:
  import fiona
except:
  !pip install fiona
  import fiona
from shapely.geometry import shape

### Defining parameters for data discovering / processing

In [2]:
#@title Upload shapefile over AoI (zip format)

myupload = widgets.FileUpload(
    accept= '.zip', 
    multiple=False  
)
myupload

FileUpload(value={}, accept='.zip', description='Upload')

In [3]:
#@title Select start date from the calendar
startdate=widgets.DatePicker(
    description='Start Date',
    disabled=False
)
startdate

DatePicker(value=None, description='Start Date')

In [4]:
#@title Select stop date as DAY AFTER THE EVENT
stopdate=widgets.DatePicker(
    description='Event Date',
    disabled=False
)
stopdate

DatePicker(value=None, description='Event Date')

# Installing software and preparing system

## Installing Sentinelsat for Sentinel data download

In [ ]:
#@title Installing sentinelsat
!pip install sentinelsat
from datetime import date

In [ ]:
#@title Obtaining footprint from shapefile
## Discovering selected file
#latmin=None
#lonmin=None
#latmax=None
#lonmax=None
if len(myupload.value)>0:
  print(myupload.value)
  uploaded_filename = next(iter(myupload.value))
  content = myupload.value[uploaded_filename]['content']
  filename=myupload.metadata[0]['name']
  print('#############################################')
  print('Selected file: '+filename)
  # Uploading file in notebook / server
  print('#############################################')
  print('Uploading file: '+filename)
  with open(filename, 'wb') as f: f.write(content)
  ## Unzipping file
  print('#############################################')
  print('Unzipping file: '+filename)
  with zipfile.ZipFile(filename, 'r') as zip_ref:
      zip_ref.extractall(os.getcwd())

  ## Finding file and reading coordinates
  print('#############################################')
  print('Extracting Bounding Box from shapefile')
  shpfile=glob.glob(os.path.join(os.getcwd(),'*.shp'))
  print(shpfile)
  c = fiona.open(shpfile[0])
  #print(c.bounds)
  lonmin,latmin,lonmax,latmax=c.bounds
  #print(lonmin)
  #print(latmin)
  footprint='POLYGON(('+str(lonmin)+' '+str(latmin)+','+str(lonmin)+' '+str(latmax)+','+str(lonmax)+' '+str(latmax)+','+str(lonmax)+' '+str(latmin)+','+str(lonmin)+' '+str(latmin)+'))'
  print('Shapefile footprint')
  print(footprint)
else:
  while latmin is None or float(latmin)>90 or float(latmin)<-90:
    print('Introduce latmin coordinate (decimal degrees)')
    latmin=input()
  while latmax is None or float(latmax)<=float(latmin) or float(latmax)>90 or float(latmax)<-90:
    print('Introduce latmax coordinate (decimal degrees)')
    latmax=input()
  while lonmin is None or float(lonmin)<-180 or float(lonmin)>180:
    print('Introduce lonmin coordinate (decimal degrees)')
    lonmin=input()
  while lonmax is None or float(lonmax)<=float(lonmin) or float(lonmax)<-180 or float(lonmax)>180:
    print('Introduce lonmax coordinate (decimal degrees)')
    lonmax=input()
  print('Bounding Box defined by:')
  print(latmin,latmax,lonmin,lonmax)
#######################################
# Getting startime
print(startdate)
if startdate.value is not None:
  starttime=startdate.value.strftime('%Y%m%d')
  print('#########################################')
  print('Start date: '+starttime)
else:
  print('Introduce start time in "YYYYMMDD" format')
  starttime=input()
#######################################3
## Getting stoptime
if stopdate.value is not None:
  stoptime=stopdate.value.strftime('%Y%m%d')
  print('########################################')
  print('Stop date: '+stoptime)
else:
  print('Introduce stop time in "YYYYMMDD" format')
  stoptime=input()
if starttime is not None and stoptime is not None:
  print('Time interval selected:')
  print(starttime,stoptime)
  acdate=(starttime,stoptime) 

In [7]:
#@title Select relative Orbit or track (in this example track=63)
print('Select relative orbit (track)')
track=input()

Select relative orbit (track)
63


In [8]:
#@title search parameters definition
# Defining AoI Bounding Box
#latmin=30.46
#latmax=30.50
#lonmin=79.6
#lonmax=79.70

#acdate=('20210115',date(2021,2,12))
#track=63

In [ ]:
#@title Search and download Sentinel-1 files
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from getpass import getpass
print('Introduce credentials for data download from Copernicus Open Data Hub')
print('User')
user=input()
print('Password')
password = getpass()
api = SentinelAPI(user, password)
#footprint = geojson_to_wkt(read_geojson('search_polygon.geojson'))
footprint='POLYGON(('+str(lonmin)+' '+str(latmin)+','+str(lonmin)+' '+str(latmax)+','+str(lonmax)+' '+str(latmax)+','+str(lonmax)+' '+str(latmin)+','+str(lonmin)+' '+str(latmin)+'))'
products = api.query(footprint,
                     producttype='SLC',
                     date = acdate,
                     relativeOrbitNumber=track)
                    # orbitdirection='Descending')
print(products)
# Downloading the files in /content folder
api.download_all(products)
dataset=api.to_dataframe(products)
print(dataset)

## Installing and configuring SNAP and dependencies for Satellite data processing

In [ ]:
#@title Installing packages for S1 data processing
!apt-get install -y libfftw3-dev libtiff5-dev gdal-bin gfortran libgfortran5 libgfortran3 jblas git
# Downloading the package
#!wget http://step.esa.int/downloads/8.0/installers/esa-snap_sentinel_unix_8_0.sh
#!wget https://download.esa.int/step/snap/8.0/installers/esa-snap_sentinel_unix_8_0.sh 
!cp /content/drive/MyDrive/WB_coherence/esa-snap_sentinel_unix_8_0.sh /content/.
# configuration of installation parameters for Sentinel-1 Toolbox only
!TBX=esa-snap_sentinel_unix_8_0.sh ; chmod +x $TBX ; echo -e "deleteAllSnapEngineDir\$Boolean=false\ndeleteOnlySnapDesktopDir\$Boolean=false\nexecuteLauncherWithPythonAction\$Boolean=false\nforcePython\$Boolean=false\npythonExecutable=/usr/bin/python\nsys.adminRights\$Boolean=true\nsys.component.RSTB\$Boolean=true\nsys.component.S1TBX\$Boolean=true\nsys.component.S2TBX\$Boolean=false\nsys.component.S3TBX\$Boolean=false\nsys.component.SNAP\$Boolean=true\nsys.installationDir="$PWD"/snap\nsys.languageId=en\nsys.programGroupDisabled\$Boolean=false\nsys.symlinkDir=/usr/local/bin" >$PWD/snap.varfile ; ./$TBX -q -varfile $PWD/snap.varfile
# Updating SNAP
#!$PWD/snap/bin/snap --nosplash --nogui --modules --update-all
# Refresh module list
!$PWD/snap/bin/snap --nosplash --nogui --modules --list --refresh #2>&1 | while read -r line; do echo "$line" ; [ "$line" = "-------------------------------------------------- -------- ----------------" ] && sleep 2 && pkill -TERM -f "snap/jre/bin/java"; done
# Update all modules
!$PWD/snap/bin/snap --nosplash --nogui --modules --update-all 2>&1 | while read -r line; do echo "$line" ; [ "$line" = "updates=0" ] && sleep 2 && pkill -TERM -f "snap/jre/bin/java"; done
# Configuring memory and parameters
!echo "-Xmx10G" > $PWD/snap/bin/gpt.vmoptions

#!cp /content/drive/My\ Drive/WB_Coherence/snap.properties $PWD/snap/bin/.

In [12]:
#@title write snap.properties
%%writefile ./snap/bin/snap.properties
# tileCache and tileSize jai properties affect graph processing and image rendering
# Tile cache size [Mb]
snap.jai.tileCacheSize=8196
# Default tile size in pixels
snap.jai.defaultTileSize=2048
# number of CPU cores used for image rendering and graph processing
# allow this to default to Runtime.getRuntime().availableProcessors()
#snap.parallelism = 8
# Enable or disable the prefetch of tiles. Default is true.
snap.jai.prefetchTiles=true

Writing ./snap/bin/snap.properties


In [50]:
#@title connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
#@title Copying Satellite original images from Google Drive if needed
import glob
import shutil
try:
  from tqdm.notebook import tqdm, trange
except:
  !pip install tqdm
  from tqdm.notebook import tqdm, trange
print('Total files to copy: '+str(len(glob.glob('/content/drive/MyDrive/WB_Coherence/Tapovan/results1/*zip'))))
filelist=glob.glob('/content/drive/MyDrive/WB_Coherence/Tapovan/results1/*zip')
print(filelist)
for _ in trange(len(filelist),desc='Copying files'):
  shutil.copy(filelist[_], '/content/')
  print(filelist[_]+' successfully copied!')
  #!cp /content/drive/MyDrive/WB_Coherence/Tapovan/results1/*zip .

Total files to copy: 3
['/content/drive/MyDrive/WB_Coherence/Tapovan/results1/S1A_IW_SLC__1SDV_20210118T004342_20210118T004408_036185_043E36_31C5.zip', '/content/drive/MyDrive/WB_Coherence/Tapovan/results1/S1A_IW_SLC__1SDV_20210130T004341_20210130T004408_036360_04444D_13C0.zip', '/content/drive/MyDrive/WB_Coherence/Tapovan/results1/S1A_IW_SLC__1SDV_20210211T004341_20210211T004408_036535_044A5F_DDDC.zip']


# Satellite data processing

## Defining satellite data processing parameters

In [17]:
#@title list available images
import glob
import os
images=sorted(glob.glob('/content/drive/MyDrive/WB_Coherence/Tapovan/results1/*zip'))
images=sorted(glob.glob('/content/*zip'))
for image in images:
  print(os.path.basename(image))

S1A_IW_SLC__1SDV_20210118T004342_20210118T004408_036185_043E36_31C5.zip
S1A_IW_SLC__1SDV_20210130T004341_20210130T004408_036360_04444D_13C0.zip
S1A_IW_SLC__1SDV_20210211T004341_20210211T004408_036535_044A5F_DDDC.zip
Tapovan.zip


In [18]:
#@title Defining required parameters for satellite data processing
# Sentinel-1 SLC images
import glob
import os
images=sorted(glob.glob('S1*zip'))
reference=images[1]
secondaries=[images[0],images[2]]
print(reference)
print(secondaries)

# Subswath 
IWs='IW1'
# polarisation
polarisation='VV'
# Defining AoI Bounding Box
#latmin=30.46
#latmax=30.50
#lonmin=79.6
#lonmax=79.70
if footprint is None:
  footprint='POLYGON(('+str(lonmin)+' '+str(latmax)+','+str(lonmax)+' '+str(latmax)+','+str(lonmax)+' '+str(latmin)+','+str(lonmin)+' '+str(latmin)+','+str(lonmin)+' '+str(latmax)+'))'
print(footprint)

# Defining graphs to be employed (Graph xml includes all steps to run using SNAP in command line)
outputifgfolder=os.getcwd()
graphxml=os.path.join(os.getcwd(),'WB_Coh_TC_Subset2.xml')
graph2run=os.path.join(os.getcwd(),'graph2run.xml')


S1A_IW_SLC__1SDV_20210130T004341_20210130T004408_036360_04444D_13C0.zip
['S1A_IW_SLC__1SDV_20210118T004342_20210118T004408_036185_043E36_31C5.zip', 'S1A_IW_SLC__1SDV_20210211T004341_20210211T004408_036535_044A5F_DDDC.zip']
POLYGON((79.62536653659116 30.49149746977097,79.62536653659116 30.49481998974621,79.63243054414228 30.49481998974621,79.63243054414228 30.49149746977097,79.62536653659116 30.49149746977097))


In [20]:
#@title Writing graph template for SNAP execution
%%writefile ./WB_Coh_TC_Subset2.xml 
<graph id="Graph">
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <file>REFERENCE</file>
    </parameters>
  </node>
  <node id="Read(2)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <file>SECONDARY</file>
    </parameters>
  </node>
  <node id="TOPSAR-Split">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <subswath>IWs</subswath>
      <selectedPolarisations>POLARISATION</selectedPolarisations>
      <firstBurstIndex>1</firstBurstIndex>
      <lastBurstIndex>9</lastBurstIndex>
      <wktAoi>POLYGON</wktAoi>
    </parameters>
  </node>
  <node id="TOPSAR-Split(2)">
    <operator>TOPSAR-Split</operator>
    <sources>
      <sourceProduct refid="Read(2)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <subswath>IWs</subswath>
      <selectedPolarisations>POLARISATION</selectedPolarisations>
      <firstBurstIndex>1</firstBurstIndex>
      <lastBurstIndex>10</lastBurstIndex>
      <wktAoi/>
    </parameters>
  </node>
  <node id="Apply-Orbit-File">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="TOPSAR-Split"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <polyDegree>3</polyDegree>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
  <node id="Apply-Orbit-File(2)">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="TOPSAR-Split(2)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
      <polyDegree>3</polyDegree>
      <continueOnFail>false</continueOnFail>
    </parameters>
  </node>
  <node id="Back-Geocoding">
    <operator>Back-Geocoding</operator>
    <sources>
      <sourceProduct refid="Apply-Orbit-File"/>
      <sourceProduct.1 refid="Apply-Orbit-File(2)"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <demName>SRTM 1Sec HGT</demName>
      <demResamplingMethod>BILINEAR_INTERPOLATION</demResamplingMethod>
      <externalDEMFile/>
      <externalDEMNoDataValue>0.0</externalDEMNoDataValue>
      <resamplingType>BISINC_21_POINT_INTERPOLATION</resamplingType>
      <maskOutAreaWithoutElevation>true</maskOutAreaWithoutElevation>
      <outputRangeAzimuthOffset>false</outputRangeAzimuthOffset>
      <outputDerampDemodPhase>false</outputDerampDemodPhase>
      <disableReramp>false</disableReramp>
    </parameters>
  </node>
  <node id="TOPSAR-Deburst">
    <operator>TOPSAR-Deburst</operator>
    <sources>
      <sourceProduct refid="Coherence"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <selectedPolarisations>POLARISATION</selectedPolarisations>
    </parameters>
  </node>
  <node id="Coherence">
    <operator>Coherence</operator>
    <sources>
      <sourceProduct refid="Back-Geocoding"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <cohWinAz>3</cohWinAz>
      <cohWinRg>10</cohWinRg>
      <subtractFlatEarthPhase>true</subtractFlatEarthPhase>
      <srpPolynomialDegree>5</srpPolynomialDegree>
      <srpNumberPoints>501</srpNumberPoints>
      <orbitDegree>3</orbitDegree>
      <squarePixel>true</squarePixel>
      <subtractTopographicPhase>true</subtractTopographicPhase>
      <demName>SRTM 1Sec HGT</demName>
      <externalDEMFile/>
      <externalDEMNoDataValue>0.0</externalDEMNoDataValue>
      <externalDEMApplyEGM>true</externalDEMApplyEGM>
      <tileExtensionPercent>100</tileExtensionPercent>
      <singleMaster>true</singleMaster>
    </parameters>
  </node>
  <node id="Terrain-Correction">
    <operator>Terrain-Correction</operator>
    <sources>
      <sourceProduct refid="TOPSAR-Deburst"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <demName>SRTM 1Sec HGT</demName>
      <externalDEMFile/>
      <externalDEMNoDataValue>0.0</externalDEMNoDataValue>
      <externalDEMApplyEGM>true</externalDEMApplyEGM>
      <demResamplingMethod>BILINEAR_INTERPOLATION</demResamplingMethod>
      <imgResamplingMethod>BILINEAR_INTERPOLATION</imgResamplingMethod>
      <pixelSpacingInMeter>20.0</pixelSpacingInMeter>
      <pixelSpacingInDegree>1.796630568239043E-4</pixelSpacingInDegree>
      <mapProjection>GEOGCS[&quot;WGS84(DD)&quot;, &#xd;
  DATUM[&quot;WGS84&quot;, &#xd;
    SPHEROID[&quot;WGS84&quot;, 6378137.0, 298.257223563]], &#xd;
  PRIMEM[&quot;Greenwich&quot;, 0.0], &#xd;
  UNIT[&quot;degree&quot;, 0.017453292519943295], &#xd;
  AXIS[&quot;Geodetic longitude&quot;, EAST], &#xd;
  AXIS[&quot;Geodetic latitude&quot;, NORTH]]</mapProjection>
      <alignToStandardGrid>false</alignToStandardGrid>
      <standardGridOriginX>0.0</standardGridOriginX>
      <standardGridOriginY>0.0</standardGridOriginY>
      <nodataValueAtSea>true</nodataValueAtSea>
      <saveDEM>false</saveDEM>
      <saveLatLon>false</saveLatLon>
      <saveIncidenceAngleFromEllipsoid>false</saveIncidenceAngleFromEllipsoid>
      <saveLocalIncidenceAngle>false</saveLocalIncidenceAngle>
      <saveProjectedLocalIncidenceAngle>false</saveProjectedLocalIncidenceAngle>
      <saveSelectedSourceBand>true</saveSelectedSourceBand>
      <outputComplex>false</outputComplex>
      <applyRadiometricNormalization>false</applyRadiometricNormalization>
      <saveSigmaNought>false</saveSigmaNought>
      <saveGammaNought>false</saveGammaNought>
      <saveBetaNought>false</saveBetaNought>
      <incidenceAngleForSigma0>Use projected local incidence angle from DEM</incidenceAngleForSigma0>
      <incidenceAngleForGamma0>Use projected local incidence angle from DEM</incidenceAngleForGamma0>
      <auxFile>Latest Auxiliary File</auxFile>
      <externalAuxFile/>
    </parameters>
  </node>
  <node id="Subset">
    <operator>Subset</operator>
    <sources>
      <sourceProduct refid="Terrain-Correction"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <sourceBands/>
      <region>0,0,5485,1857</region>
      <referenceBand/>
      <geoRegion>POLYGON</geoRegion>
      <subSamplingX>1</subSamplingX>
      <subSamplingY>1</subSamplingY>
      <fullSwath>false</fullSwath>
      <tiePointGridNames/>
      <copyMetadata>true</copyMetadata>
    </parameters>
  </node>
  <node id="Write">
    <operator>Write</operator>
    <sources>
      <sourceProduct refid="Subset"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <file>OUTPUTIFGFOLDER/OUTPUTFILE</file>
      <formatName>GeoTIFF-BigTIFF</formatName>
    </parameters>
  </node>
  <applicationData id="Presentation">
    <Description/>
    <node id="Read">
      <displayPosition x="25.0" y="13.0"/>
    </node>
    <node id="Read(2)">
      <displayPosition x="28.0" y="229.0"/>
    </node>
    <node id="TOPSAR-Split">
      <displayPosition x="16.0" y="50.0"/>
    </node>
    <node id="TOPSAR-Split(2)">
      <displayPosition x="11.0" y="194.0"/>
    </node>
    <node id="Apply-Orbit-File">
      <displayPosition x="12.0" y="84.0"/>
    </node>
    <node id="Apply-Orbit-File(2)">
      <displayPosition x="6.0" y="160.0"/>
    </node>
    <node id="Back-Geocoding">
      <displayPosition x="116.0" y="123.0"/>
    </node>
    <node id="TOPSAR-Deburst">
      <displayPosition x="339.0" y="124.0"/>
    </node>
    <node id="Coherence">
      <displayPosition x="241.0" y="125.0"/>
    </node>
    <node id="Terrain-Correction">
      <displayPosition x="491.0" y="123.0"/>
    </node>
    <node id="Subset">
      <displayPosition x="678.0" y="126.0"/>
    </node>
    <node id="Write">
      <displayPosition x="804.0" y="124.0"/>
    </node>
  </applicationData>
</graph>


Overwriting ./WB_Coh_TC_Subset2.xml


## Launching the SNAP data processing (Coherence computation)

In [21]:
#@title Executing SNAP processing
import time
import os
import subprocess

startproc=time.perf_counter()
for secondary in secondaries:
  outputname=os.path.basename(reference)[17:25]+'_'+os.path.basename(secondary)[17:25]+'_'+IWs+'.tif'
  if not os.path.exists(outputname) and not os.path.isfile(outputname):
    with open(graphxml, 'r') as file :
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('REFERENCE',reference)
    filedata = filedata.replace('SECONDARY', secondary)
    filedata = filedata.replace('OUTPUTIFGFOLDER', outputifgfolder)
    filedata = filedata.replace('OUTPUTFILE',outputname)
    filedata = filedata.replace('POLYGON',footprint)
    filedata = filedata.replace('POLARISATION',polarisation)
    filedata = filedata.replace('IWs',IWs)
    # Write the file out again
    with open(graph2run, 'w') as file:
        file.write(filedata)
    args = [os.path.join(os.getcwd(),'snap','bin','gpt'), graph2run, '-c', '8G', '-q', '16']
    print(args)
    # Launch the processing
    #timeStarted = time.time()
    process = subprocess.Popen(args,shell=False,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)

    # Poll process.stdout to show stdout live
    while True:
      output = process.stdout.readline()
      if process.poll() is not None:
        break
      if output:
        print (output.strip().decode('utf-8'))
    rc = process.poll()

    #timeDelta = time.time() - timeStarted                     # Get execution time.
    #print('Finished process in '+str(timeDelta)+' seconds.')

    stopproc=time.perf_counter()
    proctime=(stopproc-startproc)/60
    print('Processing completed in '+str(proctime)+' minutes')

['/content/snap/bin/gpt', '/content/graph2run.xml', '-c', '8G', '-q', '16']
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Executing processing graph
INFO: org.esa.s1tbx.commons.io.ImageIOFile: Using FileCacheImageInputStream
INFO: org.esa.s1tbx.commons.io.ImageIOFile: Using FileCacheImageInputStream
INFO: org.esa.s1tbx.commons.io.ImageIOFile: Using FileCacheImageInputStream
INFO: org.esa.s1tbx.commons.io.ImageIOFile: Using FileCacheImageInputStream
INFO: org.esa.s1tbx.commons.io.ImageIOFile: Using FileCacheImageInputStream
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.snap.engine_utilities.download.DownloadableContentImpl: http retrieving http://step.esa.int/auxdata/orbits/Sentinel-1/POEORB/S1A/2021/01/S1A_OPER_AUX_POEORB_OPOD_20210218T121733_V20210128T225942_20210130T005942.EOF.zi

# Visualising results in map

In [22]:
#@title Importing needed modules
import folium
from folium import plugins
#from scipy.ndimage import imread
import glob

In [23]:
#@title definition of quicklook generation function
# function definition of quicklook generation
from datetime import datetime
import time
from subprocess import Popen,PIPE,STDOUT

def quicklook_gen(image): #,colormap):
    now = datetime.now()
    tmpfolder=os.getcwd()
    cmd=['./snap/bin/pconvert','-b','1',image,'-f','png','-o',tmpfolder]
    print('#########################')
    print('Creating browse image')
    timeStarted = time.time()
    process = Popen(cmd, stdout=PIPE, stderr=STDOUT)
    r = b""
    for line in process.stdout:
        r += line
        print(str(line))
    process.wait()
    timeDelta = time.time() - timeStarted                     # Get execution time.
    print('Finished process in '+str(timeDelta)+' seconds.')
    if process.returncode != 0 :
        message='Error creating browse image'
    else:
        message='Browse image successfully completed.'
        print( message)

In [24]:
#@title creating browse images 
images=glob.glob(os.path.join(os.getcwd(),'2*.tif'))
print(images)
for file in images:
    print('Generating quicklook of file:'+file)
    quicklook_gen(file)#,None)

['/content/20210130_20210118_IW1.tif', '/content/20210130_20210211_IW1.tif']
Generating quicklook of file:/content/20210130_20210118_IW1.tif
#########################
Creating browse image
b'INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters\n'
b'INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.\n'
b'reading file /content/20210130_20210118_IW1.tif\n'
b'INFO: org.hsqldb.persist.Logger: dataFileCache open start\n'
b"creating histogram for band 'coh_IW1_VV_30Jan2021_18Jan2021'...\n"
b'creating RGB image...\n'
b"writing RGB image to '/content/20210130_20210118_IW1.png'...\n"
Finished process in 7.278155326843262 seconds.
Browse image successfully completed.
Generating quicklook of file:/content/20210130_20210211_IW1.tif
#########################
Creating browse image
b'INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapt

In [33]:
#@title visualising function definition
def visualising(images,tag_names):
    file=images[0]
    print(file[:-4]+'.tif')
    src=rio.open(file[:-4]+'.tif')
    BBOX=src.bounds
    print(BBOX)
    ymin=BBOX[1]
    xmin=BBOX[0]
    ymax=BBOX[3]
    xmax=BBOX[2]
    ymean=(ymax+ymin)/2
    xmean=(xmax+xmin)/2

    ymean=30.493636
    xmean=79.629

    #m = folium.Map([ymean, xmean], zoom_start=20, tiles='stamenterrain',crs='EPSG3857')
    token = 'pk.eyJ1Ijoiam9zZW1hbnVlbGRlbGdhZG9ibGFzY28iLCJhIjoiY2tjdGxkcGMzMGZqMjJzbnRzN3hzZGFpNSJ9.OAiDxye2w6didAUn2DnJRw'
    # your mapbox token
    tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

    m = folium.Map(location=[ymean, xmean], zoom_start=15,tiles=None)
    folium.TileLayer(tileurl, name='myMapbox',attr='Mapbox').add_to(m)
    #merc = os.path.join(file)
    k=0
    for merc in images:
        if not os.path.isfile(merc):
            print(f'Could not find {merc}')
        else:
            img = folium.raster_layers.ImageOverlay(
                name=tag_names[k],
        #        image=band,
                image=merc,
                bounds=[[ymin, xmin], [ymax, xmax]],
                opacity=0.6,
                interactive=True,
                cross_origin=False,
                zindex=1,
            )   
            img.add_to(m)
        k=k+1
    jsonfile='/content/drive/MyDrive/WB_Coherence/Tapovan/Tapovan_Dam.geojson'
    gjson = folium.GeoJson(jsonfile,name='Tapovan Dam').add_to(m)
    
    folium.LayerControl().add_to(m)
    m.save(os.path.join(os.getcwd(), 'SNAP_S1_CCC_Tapovan.html'))
    return m

In [34]:
#@title Visualising resulting products
try:
  import rasterio as rio
except:
  !pip install rasterio
  import rasterio as rio
import folium
# Visualising images in the map
images=sorted(glob.glob(os.path.join(os.getcwd(),'*png')))
tags=['Pre-event1','Post-event']
m=visualising(images,tags)

import pandas as pd
# Adding point to map
id = [0]
name = ['Tapovan dam']
latitude = ['30.493442']
longitude = ['79.629040']
region = ['Uttarakhand (India)']
df_counters = pd.DataFrame(
    {'ID' : id,
     'Name' : name,
     'latitude' : latitude,
     'longitude' : longitude,
     'region' : region
    })
df_counters.head()
locations = df_counters[['latitude', 'longitude']]
locationlist = locations.values.tolist()
print(len(locationlist))
print(locationlist[0])

folium.Marker([30.493442, 79.629040], popup='Tapovan Dam\n'+'Uttarakhand (India)').add_to(m)

m

/content/20210130_20210118_IW1.tif
BoundingBox(left=79.62523033807757, bottom=30.49146566417301, right=79.63259652340736, top=30.494879262252663)
1
['30.493442', '79.629040']


In [82]:
#@title computing average coherence within Tapovan geojson
import glob
try:
  import rasterstats
except:
  !pip install rasterstats
  from rasterstats import zonal_stats
#import fiona
images=sorted(glob.glob('/content/drive/MyDrive/WB_Coherence/Tapovan/results1/2*tif'))
print(images)
shapefile='/content/drive/MyDrive/WB_Coherence/Tapovan/Tapovan_Dam.geojson'
#shape = fiona.open(shapefile)
for image in images:
  print(image)
  #zonal_stats(shapefile, image, stats="count min mean max median")
  result=zonal_stats(shapefile, image, stats="median")
  print(result)

['/content/drive/MyDrive/WB_Coherence/Tapovan/results1/20210130_20210118_IW1.tif', '/content/drive/MyDrive/WB_Coherence/Tapovan/results1/20210130_20210211_IW1.tif']
/content/drive/MyDrive/WB_Coherence/Tapovan/results1/20210130_20210118_IW1.tif


/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


[{'median': 0.5013766884803772}]
/content/drive/MyDrive/WB_Coherence/Tapovan/results1/20210130_20210211_IW1.tif
[{'median': 0.2735366225242615}]


## Connecting Google drive

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Saving all the results in the Google Drive folder

In [ ]:
# Copying Results to the Google Drive folder
print('Copying from Google Colab to my Google Drive')
!cp $PWD/*zip /content/drive/My\ Drive/WB_Coherence/Tapovan/results/.
print('Copy completed')

Copying from Google Colab to my Google Drive
cp: cannot create regular file '/content/drive/My Drive/WB_Coherence/Tapovan/results/./S1A_IW_SLC__1SDV_20201201T004344_20201201T004411_035485_0425F7_506D.zip': Input/output error
cp: cannot create regular file '/content/drive/My Drive/WB_Coherence/Tapovan/results/./S1A_IW_SLC__1SDV_20201213T004343_20201213T004410_035660_042BF9_3E3C.zip': Input/output error
cp: cannot create regular file '/content/drive/My Drive/WB_Coherence/Tapovan/results/./S1A_IW_SLC__1SDV_20201225T004343_20201225T004410_035835_0431FD_77CC.zip': Input/output error
cp: cannot create regular file '/content/drive/My Drive/WB_Coherence/Tapovan/results/./S1A_IW_SLC__1SDV_20210106T004342_20210106T004409_036010_043816_98BF.zip': Input/output error
cp: cannot create regular file '/content/drive/My Drive/WB_Coherence/Tapovan/results/./S1A_IW_SLC__1SDV_20210118T004342_20210118T004408_036185_043E36_31C5.zip': Input/output error
cp: cannot create regular file '/content/drive/My Drive